## Программирование для журналистики данных
Магистерская программа [«Журналистика данных»](https://www.google.ru/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=0ahUKEwim7bzM1YnQAhWBXiwKHcY2DXMQFggdMAA&url=https%3A%2F%2Fwww.hse.ru%2Fma%2Fdatajourn%2F&usg=AFQjCNG4iSK2wv3y8LuhNEfWjmP-Qu6sZA&sig2=6NhsKJRk52Evdj871uVSxw), факультет коммуникаций, медиа и дизайна, НИУ ВШЭ, 2016-17 учебный год.

- Автор курса: Илья Щуров.
- [Страница курса](http://math-info.hse.ru/s16/5)
- Этот ноутбук доступен по лицензии [CC BY SA 4.0 Unported](http://creativecommons.org/licenses/by-sa/4.0/). Вы можете использовать его в любых целях при условии указания имени автора, аффилиации и ссылки на страницу курса. Все фрагменты кода также распространяются по лицензии MIT.

## Занятие 2

### План
#### Успели
- Функции как способ структурирования программы.
- [`args`](http://nbviewer.jupyter.org/github/ischurov/pythonhse/blob/master/Lecture%204.ipynb#Передача-аргументов) и [`kwargs`](http://nbviewer.jupyter.org/github/ischurov/pythonhse/blob/master/Lecture%206.ipynb#**kwargs);

#### Не успели
- [хитрости с вещественными числами](http://nbviewer.jupyter.org/github/ischurov/pythonhse/blob/master/Lecture%206.ipynb#Хитрости-с-вещественными-числами);
- ещё об изменяемых типах данных: `copy` vs `deepcopy`;
    - default values и изменяемые типы данных;
- [множества](http://nbviewer.jupyter.org/github/ischurov/pythonhse/blob/master/Lecture%207.ipynb#Множества).

### Задачи

#### Успели
1. В файле [electors.csv](https://github.com/ischurov/dj-prog/blob/master/elections_usa_2016/electors.csv) приведено количество выборщиков, закреплённых за каждым штатом США (по состоянию на 2016 год). В файле [results.csv](https://github.com/ischurov/dj-prog/blob/master/elections_usa_2016/results.csv) приведены искусственные результаты выборов (доля голосов за каждого кандидата по каждому штату). Считая, что все штаты инструктируют всех своих выборщиков голосовать за кандидата, набравшего большинство голосов в штате (строго говоря, это неверно: в Мэне и Небраске более сложные правила), найти победителя выборов.

#### Не успели

1. Написать функцию `median`, принимающую на вход несколько аргументов и возвращающую их медиану.
5. Написать функцию `append_column(table, column, inplace)`, принимающую на вход таблицу в виде списка списков и новую колонку и добавляющую колонку в таблицу. Если `inplace` установлен в `True`, функция должна модифицировать исходную таблицу и ничего не возвращать. Если `inplace` установлен в `False`, функция должна вернуть новую таблицу, а старую не модифицировать. Если `inplace` не задан, считать, что он задан в `False`.
6. Написать функцию `add_suffixes(strings, prefix, suffix, inplace)`, принимающую на вход список строк `strings`, строку `prefix` и строку `suffix` и приписывающую к каждой строке из `strings` `prefix` (к началу строки) и `suffix` (к концу). Если `inplace` установлен в `True`, следует модифицировать исходный список `strings` и ничего не возвращать, а если в `False`, то не модифицировать исходный список и вернуть новый список. По умолчанию считать, что `inplace` установлен в `False`.

### Решение задачи про выборы

#### Первоначальная версия
Предполагается, что мы скачали указанные в условии csv-файлы и положили их в каталог `elections_usa_2016` в текущем каталоге. (Напомним, что выяснить, в каком каталоге мы сейчас находимся, можно с помощью

    import os
    os.getcwd()

In [25]:
electors_file = open("elections_usa_2016/electors.csv")
electors_file.readline()
# первая строчка содержит данные об именах столбцов, для нашей программы
# она не нужна и мы её пропускаем: считываем и никак не используем

state_to_electors = {}
# нам нужно уметь, зная штат, определять, сколько у него выборщиков
# естественный объект для хранения таких данных: словарь, ключами
# которого являются названия штатов (они уникальны), а значениями — число
# выборщиков

for line in electors_file:
    # перебор строк в файле начнётся со второй, потому что 
    # первую мы уже считали
    
    state, num_of_electors = line.split(',')
    state_to_electors[state] = int(num_of_electors)
    # в принципе, есть более универсальные методы чтения csv-файлов,
    # но мы сейчас используем простой подход, который сработает
    # в нашем случае (но не факт, что сработает в других)
    
    # лучше пользоваться готовыми библиотеками для этого
    
electors_file.close()

# Теперь займёмся файлом с результатами по каждому штату

results_file = open("elections_usa_2016/results.csv")
header = results_file.readline()
# на этот раз нам понадобится сохранить первую строчку
# в ней указаны имена кандидатов
# если мы хотим написать программу, которая будет работать с разными входными
# данными (например, через 4 года), то нужно использовать эту информацию

candidates = header.strip().split(',')[1:]
# .strip() убирает лишние пробелы и символы перевода строк 
# из начала и конца строки
# дальше мы делим строчку по запятой (.split(','))
# получаем список
# от этого списка мы берём все элементы, кроме первого ([1:])

# теперь мы будем изображать центризбирком
# к нам будут приходить данные по каждому штату 
# мы будем определять, сколько в этом штате победителей
# дальше мы узнаем, сколько выборщиков относятся к этому штату
# и запишем этих выборщиков на счёт победителя
# нам понадобится «табло», в котором будет записано, сколько 
# выборщиков есть сейчас у каждого кандидата
# это табло разумно организовать в виде словаря
# инициализируем его нулями

sum_of_votes = {candidate:0 for candidate in candidates}

# теперь приступим к подсчёту

for line in results_file:
    row = line.strip().split(',')
    state = row[0]
    # в первом столбце написано название штата
    votes = [float(vote) for vote in row[1:]]
    # в остальных столбцах — числа с плавающей точкой, которые
    # считались как строки, но нам нужно превратить их именно в числа
    # здесь мы используем списковые включения
    # мы перебираем элементы списка row[1:], который получается из row
    # отрезанием первого элемента
    
    max_votes = max(votes)
    # находим, какую долю голосов набрал победитель
    
    winner_idx = votes.index(max_votes)
    # находим номер столбца, в котором записано имя победителя
    # нумерация начинается с нуля
    
    # теперь нужно найти имя победителя в штате
    # на этот случай у нас есть список кандидатов candidates
    # они упорядочены ровно так же, как голоса
    # поэтому нам нужно просто взять элемент с индексом winner_idx
    winner = candidates[winner_idx]
    
    # теперь нужно понять, сколько выборщиков есть у текущего штата
    # информация о том, сколько выборщиков есть у какого штата, хранится
    # в словаре state_to_electors
    # нам нужно получить значение, соответствующее ключу, равному названию
    # текущего штата
    # в этом месте многие пытались писать циклы: это неудачное решение
    # словари как раз существуют для того, чтобы быстро извлекать из них
    # значение по ключу
    
    sum_of_votes[winner] += state_to_electors[state]
    
# всё, файл обработан, его можно закрыть

electors_file.close()

# теперь нужно найти победителя по стране
# для этого нужно понять, есть ли кто-то, кто набрал 270 голосов или больше

# переберём все записи в sum_of_votes и выберем из них только те,
# в которых значение больше или равно 270
# от него нам нужно только имя кандидата
final_winner = [candidate for candidate, votes in sum_of_votes.items()
                                         if votes >= 270]
# получающийся список имеет один или ноль элементов (по крайней мере)
# если данные по выборщикам соответствуют действительности

print("And the winner is...", final_winner[0])

And the winner is... Trump


Наш код работает, но чтобы сделать понятным, что происходит на каждом шаге, нам пришлось написать кучу комментариев. Вот так выглядел бы этот код без комментариев.

In [ ]:
electors_file = open("elections_usa_2016/electors.csv")
electors_file.readline()
state_to_electors = {}
for line in electors_file:
    state, num_of_electors = line.split(',')
    state_to_electors[state] = int(num_of_electors)
electors_file.close()

results_file = open("elections_usa_2016/results.csv")
header = results_file.readline()

candidates = header.strip().split(',')[1:]
sum_of_votes = {candidate:0 for candidate in candidates}

for line in results_file:
    row = line.strip().split(',')
    state = row[0]
    votes = [float(vote) for vote in row[1:]]
    max_votes = max(votes)
    winner_idx = votes.index(max_votes)
    winner = candidates[winner_idx]
    sum_of_votes[winner] += state_to_electors[state]
electors_file.close()
final_winner = [candidate for candidate, votes in sum_of_votes.items()
print("And the winner is...", final_winner[0])

Готов поспорить, что через неделю вы не сможете быстро сказать, что делает та или иная строчка. Наша программа не структурирована. Один из способов структурировать программы (помимо того, чтобы писать много комментариев, что тоже полезно) — это разбивать решение задачи на отдельные действия и оформлять эти действия функциями.

В нашем случае можно выделить такие действия:

1. Считать файл со списком выборщиков.
2. Считать файл с результатами голосования.
3. Для каждого штата, найти, кто победил в этом штате.
4. Найти, сколько выборщиков получил каждый кандидат.
5. Найти, кто же победил.

Каждое из этих действий можно оформить в виде функции. У функций принято писать описание в длинных тройных кавычках. 

In [1]:
# шаг 1
def get_electors(electors_csv):
    """
    Reads file electors_csv
    Returns dictionary (state -> number of electors)
    """
    file = open(electors_csv)
    file.readline()
    state_to_electors = {}
    for line in file:
        state, num_of_electors = line.split(',')
        state_to_electors[state] = int(num_of_electors)
    file.close()
    return state_to_electors

In [ ]:
# шаг 2
def get_results(results_csv):
    """
    Reads csv file with results
    Returns a touple (candidates, results)
    candidates is a list of candidate names
    results is a table (list of lists),
    first row is state name
    other rows are vote shares of particular candidate
    """
    file = open(results_csv)
    header = file.readline()
    candidates = header.strip().split(',')[1:]
    table = []
    for line in file:
        row = line.strip().split(',')
        state = row[0]
        votes = [float(vote) for vote in row[1:]]
        table.append([state] + votes)
    file.close()
    return candidates, table

In [2]:
# шаг 3
def find_winner(votes):
    """
    votes is a list of vote shares (float numbers)
    returns name of a winner
    uses global name candidates (list of candidate names)
    """
    max_votes = max(votes)
    winner_idx = votes.index(max_votes)
    winner = candidates[winner_idx]
    return winner

In [23]:
# шаг 4 сделаем ниже

In [24]:
# шаг 5
def get_final_winner(sum_of_votes):
    final_winner = [candidate for candidate, votes 
                in sum_of_votes.items()
                if votes >= 270]
    if final_winner:
        return final_winner[0]
    else:
        return None

Наконец, соберём всё воедино (этот код тоже можно было бы оформить в виде функции, он соответствует шагу 4).

In [22]:
state_to_electors = get_electors("elections_usa_2016/electors.csv")
candidates, results = get_results("elections_usa_2016/results.csv")
sum_of_votes = {candidate:0 for candidate in candidates}
for state_row in results:
    winner = find_winner(state_row[1:])
    sum_of_votes[winner] += state_to_electors[state_row[0]]    
winner = get_final_winner(sum_of_votes)
print("And the winner is...", winner)

And the winner is... Trump


Глядя на эту программу, нам понятно, что делает каждая строчка (в терминах нашей исходной задачи). При необходимости, мы можем углубиться в реализацию конкретных функций, но по крайней мере мы не обязаны это делать, если хотим разобраться в общей структуре программы.

Впрочем, у нашего нового подхода есть один недостаток по сравнению с предыдущим: нам приходится считывать всё содержимое файла `results.csv` в память, а в исходном подходе можно было обрабатывать строчку за строчкой. В нашем случае разницы нет, но если бы файл был очень, очень большим (миллионы строк, например), это могло бы создать проблемы.